In [2]:
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
import json
import os
from enum import Enum
from loguru import logger
from types import SimpleNamespace
from pydantic import BaseModel, Field

load_dotenv()
# Define the OpenAI API key and model
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model = 'gpt-4o'

In [3]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str] #= Field(default_factory=list)

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "There are no participants going to the science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

print(event)